In [1]:
import gym
import random
import numpy as np
import tflearn
import matplotlib.pyplot as plt
%matplotlib inline

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
from tflearn.layers.core import input_data, dropout, fully_connected

In [3]:
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

In [4]:
LR = 1e-3
env = gym.make('CartPole-v0')

In [5]:
env.reset()

array([ 0.02833261, -0.00422314,  0.04875655, -0.0497623 ])

In [6]:
goal_steps = 500

In [7]:
score_requiremnts = 50
initial_games = 10000

In [8]:
def some_randdom_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
#             env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
#             print(observation, reward, done, info)
            if done:
                break

In [9]:
some_randdom_games_first()

In [16]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
            
            prev_observation = observation
            
            score += reward
            if done:
                break
                
        if score >= score_requiremnts:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                training_data.append([data[0], output])
        
        env.reset()
        scores.append(score)
        
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

                    

In [17]:
initial_population()

Average accepted score:  60.3048433048433
Median accepted score:  57.0
Counter({51.0: 33, 50.0: 32, 53.0: 26, 54.0: 23, 52.0: 20, 55.0: 18, 56.0: 17, 57.0: 15, 61.0: 15, 59.0: 14, 60.0: 13, 58.0: 11, 67.0: 10, 62.0: 9, 63.0: 9, 68.0: 7, 73.0: 7, 64.0: 6, 65.0: 6, 66.0: 6, 76.0: 6, 71.0: 5, 69.0: 4, 82.0: 4, 70.0: 3, 72.0: 3, 74.0: 3, 75.0: 3, 77.0: 3, 89.0: 3, 79.0: 2, 84.0: 2, 85.0: 2, 90.0: 2, 96.0: 2, 80.0: 1, 88.0: 1, 91.0: 1, 94.0: 1, 99.0: 1, 100.0: 1, 110.0: 1})


[[array([ 0.02937683, -0.14947971,  0.00624367,  0.33008363]), [1, 0]],
 [array([ 0.02638724, -0.34468998,  0.01284534,  0.62472895]), [0, 1]],
 [array([ 0.01949344, -0.1497497 ,  0.02533992,  0.33611907]), [0, 1]],
 [array([ 0.01649844,  0.04500264,  0.0320623 ,  0.05153349]), [1, 0]],
 [array([ 0.01739849, -0.15056402,  0.03309297,  0.35415742]), [0, 1]],
 [array([ 0.01438721,  0.04407213,  0.04017612,  0.0720906 ]), [1, 0]],
 [array([ 0.01526866, -0.15160209,  0.04161793,  0.37717368]), [0, 1]],
 [array([ 0.01223661,  0.04290483,  0.04916141,  0.09789808]), [1, 0]],
 [array([ 0.01309471, -0.15288598,  0.05111937,  0.40567709]), [0, 1]],
 [array([ 0.01003699,  0.0414752 ,  0.05923291,  0.12953873]), [1, 0]],
 [array([ 0.0108665 , -0.15444305,  0.06182368,  0.440305  ]), [0, 1]],
 [array([ 0.00777763,  0.03975196,  0.07062978,  0.16773358]), [0, 1]],
 [array([ 0.00857267,  0.23379557,  0.07398446, -0.10185824]), [0, 1]],
 [array([ 0.01324859,  0.42778354,  0.07194729, -0.37031259]), [

In [ ]:
def nueral_network_model(input_size):
    network